<a href="https://colab.research.google.com/github/TMUITLab/EAFR/blob/master/EA7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd '/content'
!git clone https://github.com/1049451037/GCN-Align
%cd '/content'
!git clone https://github.com/MaoXinn/RREA

/content
Cloning into 'GCN-Align'...
remote: Enumerating objects: 53, done.
remote: Total 53 (delta 0), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (53/53), done.
/content
Cloning into 'RREA'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [3]:
import warnings
warnings.filterwarnings('ignore')


from importlib.machinery import SourceFileLoader
 
layer = SourceFileLoader("layer", "/content/RREA/CIKM/layer.py").load_module()
utils = SourceFileLoader("utils", "/content/RREA/CIKM/utils.py").load_module()
CSLS = SourceFileLoader("CSLS", "/content/RREA/CIKM/CSLS.py").load_module()
import tensorflow as tf
import os
import random
import keras
from tqdm import *
import numpy as np
from utils import *
from CSLS import *
import tensorflow as tf
import keras.backend as K
from keras.layers import *
from layer import NR_GraphAttention

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [ ]:
lang = 'zh'
radj,train_pair,dev_pair,adj_matrix,r_index,r_val,adj_features,rel_features = load_data('/content/GCN-Align/data/%s_en/'%lang,train_ratio=0.30)
train_pair_main=train_pair
adj_matrix = np.stack(adj_matrix.nonzero(),axis = 1)
rel_matrix,rel_val = np.stack(rel_features.nonzero(),axis = 1),rel_features.data
ent_matrix,ent_val = np.stack(adj_features.nonzero(),axis = 1),adj_features.data

38960 3024


In [ ]:
entity1, rel1, triples1 = load_triples('/content/GCN-Align/data/%s_en/'%lang + 'triples_1')
num_entity_1 = len(entity1)
num_rel_1 = len(rel1)

In [ ]:
layer = SourceFileLoader("layer", "/content/RREA/CIKM/layer.py").load_module()

from layer import NR_GraphAttention,N3,AvgClass,sim_mat,my_loss
tf.keras.backend.clear_session()

node_size = adj_features.shape[0]
rel_size = rel_features.shape[1]
triple_size = len(adj_matrix)
batch_size = node_size


class TokenEmbedding(keras.layers.Embedding):
    """Embedding layer with weights returned."""

    def compute_output_shape(self, input_shape):
        return self.input_dim, self.output_dim

    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, inputs):
        return self.embeddings
    
def get_sim_mat(lr):
    inputs = [adj_matrix,r_index,r_val,rel_matrix,ent_matrix,radj,lr]
    inputs = [np.expand_dims(item,axis=0) for item in inputs]
    return get_sim.predict_on_batch(inputs)



def CSLS_test(thread_number = 16, csls=10,accurate = True):
    lr = np.array(dev_pair)
    [sim,_,_] = get_sim_mat(lr)
    eval_alignment_by_mat(sim, [1, 5, 10], thread_number, csls=csls, accurate=accurate)
    return None

def get_train_set(batch_size = batch_size):
    negative_ratio =  batch_size // len(train_pair) + 1
    train_set = np.reshape(np.repeat(np.expand_dims(train_pair,axis=0),axis=0,repeats=negative_ratio),newshape=(-1,2))
    np.random.shuffle(train_set); train_set = train_set[:batch_size]
    train_set = np.concatenate([train_set,np.random.randint(0,node_size,[train_set.shape[0],3])],axis = -1)
    return train_set

def get_train_set1(batch_size = batch_size):
    train_set = train_pair
    np.random.shuffle(train_set);
    train_set = np.concatenate([train_set,np.random.randint(0,node_size,train_set.shape)],axis = -1)
    return train_set

def get_trgat(node_size,rel_size,node_hidden,rel_hidden,triple_size,n_attn_heads = 2,dropout_rate = 0,gamma = 3,lr = 0.005,depth = 2):
    adj_input = Input(shape=(None,2))
    index_input = Input(shape=(None,2),dtype='int64')
    val_input = Input(shape = (None,))
    rel_adj = Input(shape=(None,2))
    ent_adj = Input(shape=(None,2))
    radj = Input(shape=(None,3),dtype='int64')
    alignment_input = Input(shape=(None,5),dtype = "int32") 
    LR = Input(shape=(None,2),dtype = "int32") 


    ent_emb = TokenEmbedding(node_size,node_hidden,trainable = True)(val_input) 
    rel_emb = TokenEmbedding(rel_size,node_hidden,trainable = True)(val_input)
    
    # E = TokenEmbedding(node_hidden,node_hidden,trainable = True)(val_input)
    # R = TokenEmbedding(node_hidden,node_hidden,trainable = True)(val_input)

    # avg = AvgClass(node_size,rel_size)
    # ent_feature,rel_feature = avg([ent_emb,rel_emb,radj])
    
    def avg(tensor,size):
        adj = K.cast(K.squeeze(tensor[0],axis = 0),dtype = "int64")   
        adj = tf.SparseTensor(indices=adj, values=tf.ones_like(adj[:,0],dtype = 'float32'), dense_shape=(node_size,size)) 
        adj = tf.compat.v1.sparse_softmax(adj) 
        l_adj = tf.compat.v1.sparse_tensor_dense_matmul(adj,tensor[1])
        return l_adj

    rel_feature = Lambda(avg,arguments={'size':rel_size})([rel_adj,rel_emb])
    ent_feature = Lambda(avg,arguments={'size':node_size})([ent_adj,ent_emb])


    encoder1 = NR_GraphAttention(node_size,activation="relu",
                                       rel_size = rel_size,
                                       depth = 1,
                                       attn_heads=n_attn_heads,
                                       triple_size = triple_size,
                                       attn_heads_reduction='average',   
                                       dropout_rate=dropout_rate)
  
    encoder2 = NR_GraphAttention(node_size,activation="relu",
                                    rel_size = rel_size,
                                    depth = 1,
                                    attn_heads=n_attn_heads,
                                    triple_size = triple_size,
                                    attn_heads_reduction='average',   
                                    dropout_rate=dropout_rate)
    sim_layer = sim_mat()
    loss_layer = my_loss(dropout_rate)
        
    opt1 = [rel_emb,adj_input,index_input,val_input,radj]
    weight =  tf.Variable(1 * [-0.5] + (6) * [1.0], trainable=False)
    weight = 5 * tf.math.softmax(weight)

    f = weight[0] * ent_emb
    loss = loss_layer([f ,alignment_input])
    s = sim_layer([f,LR])

    f = weight[1] * ent_feature
    loss += loss_layer([f ,alignment_input])
    s += sim_layer([f,LR])
    
    middle_F = encoder1([ent_feature] + opt1)
    f = weight[2] * middle_F 
    loss += loss_layer([f ,alignment_input])
    s += sim_layer([f,LR])

    middle_F = encoder2([middle_F] + opt1)
    f = weight[3] * middle_F 
    loss += loss_layer([f ,alignment_input])
    s += sim_layer([f,LR])

    f = weight[4] * rel_feature
    loss += loss_layer([f ,alignment_input])
    s += sim_layer([f,LR])
    
    middle_F = encoder1([rel_feature] + opt1)
    f = weight[5] * middle_F 
    loss += loss_layer([f ,alignment_input])
    s += sim_layer([f,LR])

    middle_F = encoder2([middle_F] + opt1)
    f = weight[6] * middle_F 
    loss += loss_layer([f ,alignment_input])
    s += sim_layer([f,LR])

    LR_norm = tf.matmul(tf.sqrt(s[1]),tf.transpose(tf.sqrt(s[2])))
     
    sim = s[0]/LR_norm
    print(loss.shape)
    loss = K.relu(gamma + loss[0]) + K.relu(gamma + loss[1])+ K.relu(gamma + loss[2])+ K.relu(gamma + loss[3])
    loss = tf.compat.v1.reduce_sum(loss,keep_dims=True) /(batch_size) 
   
    inputs = [adj_input,index_input,val_input,rel_adj,ent_adj,radj]
    train_model = keras.Model(inputs = inputs + [alignment_input],outputs = loss)   
    train_model.compile(loss=lambda y_true,y_pred: y_pred,optimizer=tf.keras.optimizers.RMSprop(lr=lr))

    feature_model = keras.Model(inputs = inputs+ [LR],outputs = [sim,s[1],s[2]])
    return train_model,feature_model


In [ ]:
model,get_sim = get_trgat(dropout_rate=0.30,node_size=node_size,rel_size=rel_size,n_attn_heads = 1,depth=2,gamma =3,node_hidden=100,rel_hidden = 100,triple_size = triple_size)
model.summary(); initial_weights = model.get_weights()

AttributeError: ignored

In [ ]:
train_pair = train_pair_main
tf.keras.backend.clear_session()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True  
sess = tf.compat.v1.Session(config=config) 

rest_set_1 = [e1 for e1, e2 in dev_pair]
rest_set_2 = [e2 for e1, e2 in dev_pair]
np.random.shuffle(rest_set_1)
np.random.shuffle(rest_set_2)

epoch = 1200
for turn in range(5):
    print("iteration %d start."%turn)
    for i in trange(epoch):
        train_set = get_train_set()
        inputs = [adj_matrix,r_index,r_val,rel_matrix,ent_matrix,radj,train_set]
        inputs = [np.expand_dims(item,axis=0) for item in inputs]
        model.train_on_batch(inputs,np.zeros((1,1)))
        if i%100 == 99:
            CSLS_test()
           
    new_pair = []
    l = np.array(rest_set_1).reshape(-1,1)
    r = np.array(rest_set_2).reshape(-1,1)
    
    [sim,_,_]  = get_sim_mat(np.concatenate([l,r],1))
    A,_ = eval_alignment_by_mat(sim, [1, 5, 10], 16,10,True,False)
    B,_ = eval_alignment_by_mat(np.transpose(sim),[1, 5, 10], 16,10,True,False)
    A = sorted(list(A)); B = sorted(list(B))
    for a,b in A:
        if  B[b][1] == a:
            new_pair.append([rest_set_1[a],rest_set_2[b]])
    print("generate new semi-pairs: %d." % len(new_pair))
    train_pair = np.concatenate([train_pair,np.array(new_pair)],axis = 0)
    for e1,e2 in new_pair:
        if e1 in rest_set_1:
            rest_set_1.remove(e1) 

    for e1,e2 in new_pair:
        if e2 in rest_set_2:
            rest_set_2.remove(e2) 

iteration 0 start.


  8%|▊         | 102/1200 [00:19<17:42,  1.03it/s]

accurate results: hits@[1, 5, 10] = [0.04 0.06 0.12], mr = 5258.603, mrr = 0.001, time = 7.928 s 


 17%|█▋        | 202/1200 [00:35<15:26,  1.08it/s]

accurate results: hits@[1, 5, 10] = [0.03 0.05 0.14], mr = 5263.836, mrr = 0.001, time = 7.759 s 


 23%|██▎       | 274/1200 [00:42<02:22,  6.52it/s]


KeyboardInterrupt: ignored

In [ ]:
(a,b) = [tf.constant([0]),tf.constant([0])]
(a,b) += [tf.constant([0]),tf.constant([0])]


SyntaxError: ignored

In [ ]:
l = np.array(rest_set_1).reshape(-1,1)
r = np.array(rest_set_2).reshape(-1,1)
[sim,L,R] = get_sim_mat(np.concatenate([l,r],1))
print(sim.shape,sim)


(10500, 10500) [[ 0.13218588  0.05917985  0.1306869  ...  0.10307832  0.25313294
   0.20482811]
 [ 0.09945321 -0.0033827   0.22579445 ...  0.16366687  0.23503412
   0.22260292]
 [ 0.08662807  0.01648146  0.23262902 ...  0.12311409  0.24079126
   0.20877074]
 ...
 [ 0.15729168  0.1390887   0.12876606 ...  0.19416602  0.20447268
   0.13953726]
 [ 0.20249043  0.11424626  0.17315485 ...  0.20799868  0.17705308
   0.19259618]
 [ 0.16082028  0.11769021  0.2659137  ...  0.24568443  0.3067721
   0.26965457]]


In [ ]:
sim

array([[ 0.13218588,  0.05917985,  0.1306869 , ...,  0.10307832,
         0.25313294,  0.20482811],
       [ 0.09945321, -0.0033827 ,  0.22579445, ...,  0.16366687,
         0.23503412,  0.22260292],
       [ 0.08662807,  0.01648146,  0.23262902, ...,  0.12311409,
         0.24079126,  0.20877074],
       ...,
       [ 0.15729168,  0.1390887 ,  0.12876606, ...,  0.19416602,
         0.20447268,  0.13953726],
       [ 0.20249043,  0.11424626,  0.17315485, ...,  0.20799868,
         0.17705308,  0.19259618],
       [ 0.16082028,  0.11769021,  0.2659137 , ...,  0.24568443,
         0.3067721 ,  0.26965457]], dtype=float32)

In [ ]:
S = sim/np.sqrt(L * np.transpose(R))

In [ ]:
eval_alignment_by_mat(S, [1, 5, 10], 16,10,True,False)